# 🧠 AI Agents Course – Section 1
## 📌 Notebook: Using an enhanced Agent with tools, using Google GenAI SDK

---

### 🚀 Objective:
Show how adding a tool (function) allows an agent to compute reliably, overcoming LLM limitations.

---

_Hassan Tabbal • Introduction to AI Agents Development Training — All rights reserved_

In [ ]:
%pip install -U -q "google-genai>=1.19.0"

In [ ]:
# If you are running this in Google Colab, you can set your Google API key as follows:
# 1. Locate the icon that looks like a key in the left sidebar of your Colab notebook.
# 2. Click on it to open the "User Data" section.
# 3. Add a new key-value pair with the key as 'GOOGLE_API_KEY' and your actual Google API key as the value.
# 4. Run this cell to retrieve the API key.
# Note: Make sure to replace 'GOOGLE_API_KEY' with your actual API key.
# from google.colab import userdata
# GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
# If you are using Github codespaces, add your API Key as a secret in your dev environment.

# GOOGLE_API_KEY = "YOUR_GOOGLE_API_KEY"  # Replace with your actual Google API key if not set in Colab
import os
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
MODEL_ID="gemini-2.5-flash"

Now Let's define a tool that the agent can use to determine today's date

In [ ]:
def get_Date()->str:
    """Gets the current date and time.

    Args:
        No arguments.

    Returns:
        A string representint the current date and time.
    """
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

Now that the tool is defined, let's retry our example

In [ ]:
from IPython.display import Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents="What's today's date? and what's the time? Also give it to me in Hijri calendar",
    config=types.GenerateContentConfig(
        tools=[get_Date],
    ),
)


Markdown(response.text)


Notice the difference between running the LLM without tools, and invoking it with tools to give external context. 
Tools not only adds context, it can be used to read data from databases or perform actions on external system. 

Let's continue by implementing the Get Weather tool. For this tool, we will use a free api key from https://www.weatherapi.com


In [ ]:
#WEATHER_API_KEY = ""  # Replace with your actual weather API key or make sure it's set in your environment

WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")

In [ ]:
import json
from datetime import datetime
def get_weather(location: str,date:str) -> str:
    """Gets the weather for a given location on a particular date.

    Args:
        location (str): The location for which to get the weather.
        date (str): The date for which to get the weather forecast in 'YYYY-MM-DD' format.

    Returns:
        A string representing the json dumps of the weather information as returned by the API.
    """
    import requests
    # if the date is today, we will get the current weather using the 'current' endpoint. 
    # if the date is in the future (14 days from now), we will get the forecast using the 'future' endpoint.
    # if the date is in the past (up to 7 days in the past), we will get the historical weather using the 'history' endpoint.
    # Currently, weather forecast is not available in the free tier of the API. 
    parsed_date = datetime.strptime(date, '%Y-%m-%d').date()
    today_date = datetime.today().date()
    print(f"Parsed date: {parsed_date}, Today date: {today_date}")
    WEATHER_API_URL = "https://api.weatherapi.com/v1/current.json"
    if parsed_date == today_date:  # Check if the date is today
        WEATHER_API_URL = "https://api.weatherapi.com/v1/current.json"
    elif parsed_date > today_date:  # Check if the date is in the future
        WEATHER_API_URL = "https://api.weatherapi.com/v1/future.json"
    else:  # The date is in the past
        WEATHER_API_URL = "https://api.weatherapi.com/v1/history.json"
    params = {
        'key': WEATHER_API_KEY,
        'q': location,
        'dt': date,  # Use the date parameter for historical or future weather
        'aqi': 'no'
    }
    print(f"Requesting weather data for {location} on {date} from {WEATHER_API_URL} with params: {params}")
    response = requests.get(WEATHER_API_URL, params=params)
    returnStr = ""
    if response.status_code == 200:
        data = response.json()
        #return f"The current weather in {location} is {data['current']['condition']['text']} with a temperature of {data['current']['temp_c']}°C."
        returnStr =  json.dumps(data, indent=4)  # Return the JSON data as a formatted string
    else:
        returnStr =  "Could not retrieve weather data."
    
    print(f"Weather API Response: {returnStr}")
    return returnStr
    

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="Get today's weather in Tripoli, Lebanon",
    config=types.GenerateContentConfig(
        tools=[get_Date,get_weather],
    ),
)


Markdown(response.text)

*Exercise*

Now it's your turn : Teach the LLM how to get accurate currency exchange rates using a free API key from https://freecurrencyapi.com/

*Bonus Exercise* 
Teach the LLM how to convert date to Hijri
